In [1]:
import pandas as pd
import re
import networkx as nx
from pyvis.network import Network
from itertools import combinations

In [2]:
df = pd.read_csv("ch3_regex_sample_v2.csv")
df.head()

,accident_id,memo
0,1,- 운전자 : 본인\n- 사고장소 : 화곡3동\n- 사고경위 : 좌회전신호 받고 차...
1,2,- 운전자 : 본인 \n- 사고장소 : 12:54 \n- 사고경위 : 인천공항 주차...
2,3,- 운전자 : 본인\n- 사고장소 : 17:20 홍제3동\n- 사고경위 : 주차된 ...
3,4,- 대차정보 : 김그린 010-0000-0003 / 동승자-배우자 기블루 \n#대인...
4,5,- 운전자 : 본인 (동승자 1인)\n- 사고장소 : 삼방동\n- 사고경위 : 자차...


In [3]:
# 전화번호 추출
re.findall('[0-9]{3}-?[0-9]{4}-?[0-9]{4}', df.memo[1])

['010-0000-0013', '010-0000-0007']

In [4]:
# '보험', '현장' 등이 포함된 문장은 전화번호 추출에서 제거 (노이즈 제거)
# memo 에서 문자열을 찾는 함수를 작성합니다
def find_number(memo):
    result = []
    for sentence in memo.split('\n'):

        if '보험' in sentence or '현장' in sentence:  # 보험 직원, 현장 출동 기사의 정보는 제외합니다.
            continue
        else:
            extracted_numbers = re.findall('[0-9]{3}-?[0-9]{4}-?[0-9]{4}', sentence) # 번호를 리스트 형식으로 받음
            extracted_numbers = list(map(lambda x: x.replace('-', ''), extracted_numbers)) # 번호의 - 를 없애는 작업
            result.append(extracted_numbers[0]) if len(extracted_numbers) > 0 else '' # 길이가 0보다 크면 추가, 아니면 공백 추가

    return result


df['number_involved'] = df['memo'].apply(lambda x: find_number(x))

In [5]:
df.head()

,accident_id,memo,number_involved
0,1,- 운전자 : 본인\n- 사고장소 : 화곡3동\n- 사고경위 : 좌회전신호 받고 차...,[]
1,2,- 운전자 : 본인 \n- 사고장소 : 12:54 \n- 사고경위 : 인천공항 주차...,"[01000000013, 01000000007]"
2,3,- 운전자 : 본인\n- 사고장소 : 17:20 홍제3동\n- 사고경위 : 주차된 ...,[]
3,4,- 대차정보 : 김그린 010-0000-0003 / 동승자-배우자 기블루 \n#대인...,[01000000003]
4,5,- 운전자 : 본인 (동승자 1인)\n- 사고장소 : 삼방동\n- 사고경위 : 자차...,[01000000098]


### 전화번호 조합 만들기
* 사람이 노드가 되고, 두 사람이 같은 사고에 연결되어 있다면 두 사람은 엣지(관계)를 갖는다고 할 수 있음.

In [6]:
# combination 을 이용하여 번호들의 조합을 만듭니다.
def make_combination(involved_number):
    return list(combinations(involved_number, 2))


df['number_combinations'] = df['number_involved'].apply(lambda x: make_combination(x))

In [7]:
df.tail()

,accident_id,memo,number_involved,number_combinations
31,32,- 운전자 : 본인 \n- 사고장소 : 12:54 \n- 사고경위 : 인천공항 주차...,"[01000000009, 01000000007]","[(01000000009, 01000000007)]"
32,33,- 운전자 : 김용산(01000000050)\n- 사고장소 : 용산구\n- 사고경위...,"[01000000050, 01000000034, 01000000035]","[(01000000050, 01000000034), (01000000050, 010..."
33,34,- 운전자 : 김용산(01000000070)\n- 사고장소 : 용인 \n- 사고경위...,"[01000000070, 01000000071]","[(01000000070, 01000000071)]"
34,35,- 운전자 : 김용산(01000000072)\n- 사고장소 : 용산구\n- 사고경위...,"[01000000072, 01000000073]","[(01000000072, 01000000073)]"
35,36,- 운전자 : 본인 (동승자 1인)\n- 사고장소 : 삼방동\n- 사고경위 : 자차...,"[01000000050, 01000000009]","[(01000000050, 01000000009)]"


In [8]:
# list 에 pair 를 담습니다.
numbers_pair = []
for idx, row in df.iterrows():
    for comb in row['number_combinations']:
        numbers_pair.append(list(comb))

print(numbers_pair[:10])

[['01000000013', '01000000007'], ['01000000008', '01000000004'], ['01000000008', '01000000010'], ['01000000004', '01000000010'], ['01000000011', '01000000012'], ['01000000011', '01000000013'], ['01000000012', '01000000013'], ['01000000014', '01000000015'], ['01000000014', '01000000016'], ['01000000015', '01000000016']]


In [10]:
# dataframe 을 생성합니다.
network_df = pd.DataFrame(numbers_pair, columns=['p1', 'p2'])
network_df['freq'] = 1
network_df

,p1,p2,freq
0,01000000013,01000000007,1
1,01000000008,01000000004,1
2,01000000008,01000000010,1
3,01000000004,01000000010,1
4,01000000011,01000000012,1
5,01000000011,01000000013,1
6,01000000012,01000000013,1
7,01000000014,01000000015,1
8,01000000014,01000000016,1
9,01000000015,01000000016,1


In [11]:
# 2회 연루된 사람에게는 weight 을 더 크게 줍니다.
network_df = network_df.groupby(by=['p1','p2'], as_index=False).sum('Weight').sort_values(by='freq', ascending=False )
network_df

,p1,p2,freq
5,01000000011,01000000013,2
21,01000000034,01000000035,2
0,01000000004,01000000010,1
14,01000000019,01000000020,1
25,01000000070,01000000071,1
24,01000000050,01000000035,1
23,01000000050,01000000034,1
22,01000000050,01000000009,1
20,01000000033,01000000035,1
19,01000000033,01000000034,1


### 시각화 하기

In [12]:
g = Network(notebook=True)

for index, row in network_df.iterrows():
    g.add_node(row['p1'])
    g.add_node(row['p2'])
    g.add_edge(row['p1'], row['p2'], value=row['freq'], title='freq')

In [13]:
from IPython.core.display import display, HTML

g.show('result.html')
display(HTML('result.html'))

result.html


C:\Users\김영준\AppData\Local\Temp\ipykernel_8444\869169897.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
